<a href="https://colab.research.google.com/github/shuodeng521-sys/ST-554-Project1-Shuo-Anna-Jillian/blob/main/Task1/Project1_Task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Title: Project 1 Task 1   
Class: ST-554 Big Data  
Date: 2/20/2026  
Author: Anna Giczewska

## Task 1

This task involves writing two gradient descent type algorithms to find the optimal constant to use for
squared error loss (ends up being the sample mean) and to find the optimal intercept and slope from a
simple linear regression model.


## Introduction

Monitoring carcinogenic air pollutants is important for protecting public health, but high-quality sensing equipment is often expensive and difficult to deploy widely. Prior work by De Vito et al. (2008) suggests that low-cost gas sensors may be calibrated using statistical models to approximate true pollutant concentrations.

In this task, I focus on modeling the true benzene concentration C6H6 (GT) using simple and multiple linear regression. The goal is to evaluate how well sensor and weather variables can predict benzene levels, and to compare model performance using a rolling one-step-ahead cross-validation approach.

In [ ]:
#Read data

!pip install ucimlrepo # Install UCI package - only needs to be done 1st time

In [ ]:
# Import modules
import pandas as pd
import ucimlrepo as uci
import numpy as np

In [ ]:
# Fetch the Air Quality Multisensor dataset
air_quality = uci.fetch_ucirepo(id=360)

In [ ]:
# Print var info
air_quality.variables

,name,role,type,demographic,description,units,missing_values
0,Date,Feature,Date,None,None,None,no
1,Time,Feature,Categorical,None,None,None,no
2,CO(GT),Feature,Integer,None,True hourly averaged concentration CO in mg/m^...,mg/m^3,no
3,PT08.S1(CO),Feature,Categorical,None,hourly averaged sensor response (nominally CO...,None,no
4,NMHC(GT),Feature,Integer,None,True hourly averaged overall Non Metanic Hydro...,microg/m^3,no
5,C6H6(GT),Feature,Continuous,None,True hourly averaged Benzene concentration in...,microg/m^3,no
6,PT08.S2(NMHC),Feature,Categorical,None,hourly averaged sensor response (nominally NMH...,None,no
7,NOx(GT),Feature,Integer,None,True hourly averaged NOx concentration in ppb...,ppb,no
8,PT08.S3(NOx),Feature,Categorical,None,hourly averaged sensor response (nominally NOx...,None,no
9,NO2(GT),Feature,Integer,None,True hourly averaged NO2 concentration in micr...,microg/m^3,no


In [ ]:
#Remove any observations where the C6H6(GT) or CO(GT) are -200 as these represent missing values (which we’ll ignore).

# Get the features dataframe
df = air_quality.data.features.copy()

# How many -200s you have before filtering
missing_counts = (df[['C6H6(GT)', 'CO(GT)']] == -200).sum()
print("Counts of -200 before filtering:\n", missing_counts)

# Remove rows where either column is -200
df_clean = df[(df['C6H6(GT)'] != -200) & (df['CO(GT)'] != -200)].copy()

print("\nRows before:", len(df))
print("Rows after: ", len(df_clean))
print("Removed:    ", len(df) - len(df_clean))

df_clean.head()


Counts of -200 before filtering:
 C6H6(GT)     366
CO(GT)      1683
dtype: int64

Rows before: 9357
Rows after:  7344
Removed:     2013


,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888


In [ ]:
#Assign clean version of the data frame back to df
df=df_clean

## Prediction of C6H6(GT)

In [ ]:
# Helper: RMSE for a constant c
def rmse_constant(y, c):
    return np.sqrt(np.mean((y - c)**2))

### Grid search for best constant c using only y

In [ ]:
def grid_search_best_c(y, grid_size=1000):
    q1, q3 = np.quantile(y, [0.25, 0.75])
    c_grid = np.linspace(q1, q3, grid_size)

    rmse_vals = [rmse_constant(y, c) for c in c_grid]
    best_idx = int(np.argmin(rmse_vals))
    return c_grid[best_idx], rmse_vals[best_idx], (q1, q3)

# Run for C6H6(GT)
y_c6h6 = df["C6H6(GT)"].to_numpy()
best_c, best_rmse, (q1, q3) = grid_search_best_c(y_c6h6)
print("C6H6(GT): best c =", best_c, "best RMSE =", best_rmse, "grid from", (q1, q3))

C6H6(GT): best c = 10.280180180180182 best RMSE = 7.440562418757215 grid from (np.float64(4.6), np.float64(14.3))


### Check: run the same function but with PT08.S1(CO) as y

In [ ]:
y_pt = df["PT08.S1(CO)"].to_numpy()
best_c2, best_rmse2, (q1_2, q3_2) = grid_search_best_c(y_pt)
print("PT08.S1(CO): best c =", best_c2, "best RMSE =", best_rmse2, "grid from", (q1_2, q3_2))

PT08.S1(CO): best c = 1110.5645645645645 best RMSE = 218.6664431268348 grid from (np.float64(946.0), np.float64(1246.0))


### Grid search for best b0, b1 (predict C6H6 from PT08.S1(CO))

In [ ]:
def grid_search_best_b0_b1(x, y):
    b0_grid = np.arange(-25, -15 + 0.0001, 0.1)   # -25 to -15 by 0.1
    b1_grid = np.arange(-5, 5 + 0.0001, 0.01)     # -5 to 5 by 0.01

    # Precompute means
    Ey2  = np.mean(y**2)
    Ey   = np.mean(y)
    Exy  = np.mean(x*y)
    Ex   = np.mean(x)
    Ex2  = np.mean(x**2)

    # Build a 2D grid of MSE values using broadcasting
    B0 = b0_grid[:, None]        # shape (len(b0), 1)
    B1 = b1_grid[None, :]        # shape (1, len(b1))

    mse = (Ey2
           - 2*B0*Ey
           - 2*B1*Exy
           + (B0**2)
           + 2*B0*B1*Ex
           + (B1**2)*Ex2)

    rmse = np.sqrt(mse)

    idx = np.unravel_index(np.argmin(rmse), rmse.shape)
    best_b0 = b0_grid[idx[0]]
    best_b1 = b1_grid[idx[1]]
    best_rmse = rmse[idx]

    return best_b0, best_b1, best_rmse

x = df["PT08.S1(CO)"].to_numpy()
y = df["C6H6(GT)"].to_numpy()

best_b0, best_b1, best_rmse = grid_search_best_b0_b1(x, y)
print("Best b0:", best_b0, "Best b1:", best_b1, "Best RMSE:", best_rmse)

Best b0: -22.99999999999997 Best b1: 0.02999999999989278 Best RMSE: 3.54290539083074


### Predict C6H6(GT) for x = [946, 1075, 1246]

In [ ]:
for new_x in [946, 1075, 1246]:
    pred = best_b0 + best_b1 * new_x
    print("x =", new_x, "=> predicted C6H6(GT) =", pred)

x = 946 => predicted C6H6(GT) = 5.3799999998985975
x = 1075 => predicted C6H6(GT) = 9.249999999884764
x = 1246 => predicted C6H6(GT) = 14.379999999866428


### Gradient descent for best constant c

In [ ]:
def diff_quotient_c(y, c, delta):
    return (rmse_constant(y, c + delta) - rmse_constant(y, c)) / delta

def gradient_descent_c(y, start_c, step_size, delta=0.001, tol=0.0001, max_iter=20000):
    cur = start_c
    for i in range(max_iter):
        slope = diff_quotient_c(y, cur, delta)
        new = cur - slope * step_size

        if abs(new - cur) < tol:
            return new, rmse_constant(y, new), i+1

        cur = new

    return cur, rmse_constant(y, cur), max_iter  # if it hits max_iter

# C6H6(GT) use start 0
gd_c, gd_rmse, iters = gradient_descent_c(y_c6h6, start_c=0, step_size=0.01, delta=0.001, tol=0.0001, max_iter=20000)
print("GD C6H6: c =", gd_c, "RMSE =", gd_rmse, "iters =", iters)

# PT08.S1(CO) use start 1100
gd_c2, gd_rmse2, iters2 = gradient_descent_c(y_pt, start_c=1100, step_size=0.1, delta=0.001, tol=0.0001, max_iter=30000)
print("GD PT08.S1(CO): c =", gd_c2, "RMSE =", gd_rmse2, "iters =", iters2)

GD C6H6: c = 10.200993520073336 RMSE = 7.440936478911602 iters = 3966
GD PT08.S1(CO): c = 1110.3616956683077 RMSE = 218.66655224571102 iters = 8483


### Gradient descent for best b0, b1

In [ ]:
def rmse_b0_b1(x, y, b0, b1):
    return np.sqrt(np.mean((y - (b0 + b1*x))**2))

def diff_quotient_b0(x, y, b0, b1, delta0):
    return (rmse_b0_b1(x, y, b0 + delta0, b1) - rmse_b0_b1(x, y, b0, b1)) / delta0

def diff_quotient_b1(x, y, b0, b1, delta1):
    return (rmse_b0_b1(x, y, b0, b1 + delta1) - rmse_b0_b1(x, y, b0, b1)) / delta1

def gradient_descent_b0_b1(x, y,
                           start_b0=-20, start_b1=0,
                           step_b0=0.5, step_b1=0.00005,
                           delta=0.005, tol=0.0001,
                           max_iter=100000):
    cur_b0, cur_b1 = start_b0, start_b1

    for i in range(max_iter):
        slope_b0 = diff_quotient_b0(x, y, cur_b0, cur_b1, delta)
        new_b0 = cur_b0 - slope_b0 * step_b0

        slope_b1 = diff_quotient_b1(x, y, new_b0, cur_b1, delta)
        new_b1 = cur_b1 - slope_b1 * step_b1

        move = np.linalg.norm([new_b0 - cur_b0, new_b1 - cur_b1])
        cur_b0, cur_b1 = new_b0, new_b1

        if move < tol:
            break

    return cur_b0, cur_b1, rmse_b0_b1(x, y, cur_b0, cur_b1), i+1

gd_b0, gd_b1, gd_rmse_line, gd_iters = gradient_descent_b0_b1(x, y)
print("GD best b0:", gd_b0, "GD best b1:", gd_b1, "RMSE:", gd_rmse_line, "iters:", gd_iters)

for new_x in [946, 1075, 1246]:
    pred = gd_b0 + gd_b1 * new_x
    print("x =", new_x, "=> predicted C6H6(GT) =", pred)

GD best b0: -22.304829758047333 GD best b1: -0.0014903666813404631 RMSE: 35.097272964378526 iters: 100000
x = 946 => predicted C6H6(GT) = -23.714716638595412
x = 1075 => predicted C6H6(GT) = -23.90697394048833
x = 1246 => predicted C6H6(GT) = -24.16182664299755
